In [42]:
import time

In [43]:
file = open('C:/Users/tiany/Desktop/adult.data', 'r')
data_lines = file.readlines()
data = []
for line in data_lines:
    line = line[:-1]
    data.append(line.replace(' ','').split(','))

Apriori

https://blog.csdn.net/songbinxu/article/details/80398819 gives the idea

In [53]:
def create_C1(dataset):
    """
    Goal: generate C1
    ------
    Parameter: 
        dataset: input dataset
    ------
    Return:
        C1: set with all distinct elements
    """

    C1 = set()
    for t in dataset:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1

In [76]:
support_count = {}
for itemset in data:
    for candidate in c1:
        belong = True
        tmp = list(candidate)[0].split(',')
        for i in tmp:
            if not i in itemset:
                belong = False
        if belong:
            if candidate in support_count:
                support_count[candidate] += 1
            else:
                support_count[candidate] = 1

In [18]:
def create_L(D,Ck,minSupport):
    """
    Goal: count support of item in Ck, and filter items
    ------
    Parameter: 
        D: input dataset
        Ck: C generated by either create_C1 or apriori_Gen, k indicates the length of item
        minSupport: threshold of support
    ------
    Return:
        Lk: the filtered candidate with length k, which support >= min support
        supportData: the candidate with support
    """
    support_count = {}
    for itemset in D:
        for candidate in Ck:
            if candidate.issubset(itemset):
                if candidate in support_count:
                    support_count[candidate] += 1
                else:
                    support_count[candidate] = 1
    Lk = []
    supportData = {}
    for key in support_count:
        support = support_count[key] / float(len(D))
        if support >= minSupport:
            Lk.insert(0, key)
        supportData[key] = support
    return Lk, supportData

In [82]:
def create_Ck(Lk, k):
    """
    Goal: create C set with length k
    ------
    Parameter: 
        Lk: valid items with support
        k: length of item in output Ck
    ------
    Return:
        Ck: candidate itemset, for item in Ck, length(item) = k
    """
    Ck = []
    for i in range(len(Lk)):
        for j in range(i+1,len(Lk)):
            # Check first K-2 same or not
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            print(L1,L2)
            L1.sort()
            L2.sort()
            if L1 == L2: # If same, then combine these 2 to generate new one
                Ck.append(Lk[i] | Lk[j])
    return Ck

In [21]:
def apriori(dataset, minSupport = 0.5, minConf = 0.6):
    """
    Goal: Find strong rule in dataset
    ------
    Parameter: 
        dataset: input dataset
        minSupport: min support that a item needs to satisfy
        minConf: min confidence that a rule needs to satisfy
    ------
    Return:
        L: itemset with support >= minSupport
        B: itemset with support
        rule: rule with confidence >= minConf
    """
    C1 = create_C1(dataset)
    L1, support = create_L(dataset, C1, minSupport)
    L = [L1]
    k = 2 # We already have L1, start from 2
    rule = []
    while len(L[k-2])>0:
        Ck = create_Ck(L[k-2], k)
        Lk, supportK = create_L(dataset, Ck, minSupport)
        # Each time we have a new support for length k, we will try to find the strong rule
        for i in L[len(L)-1]:
            for j in Lk:
                Anced = True # Check whether this rule is the parent of another
                for z in zip(i,j):
                    if z[0] != z[1]:
                        Anced = False
                if Anced:
                    if (supportK[j]/support[i]) >= minConf:
                        rule.append([i,j,supportK[j]/support[i]]) # Parent, Child, Conf    
        support.update(supportK)
        L.append(Lk)
        k += 1
    return L, support, rule

In [22]:
time1 = time.perf_counter()
L, C, rule = apriori(data,0.5,0.6)
time2 = time.perf_counter()
print("Running time is: ", time2 - time1)

Running time is:  0.00011840000001939188


In [8]:
rule

[[frozenset({'Private'}), frozenset({'Private', 'White'}), 0.8549524145223827],
 [frozenset({'Private'}),
  frozenset({'Private', 'United-States'}),
  0.8871607331688403],
 [frozenset({'<=50K'}), frozenset({'<=50K', 'Private'}), 0.7173543689320389],
 [frozenset({'<=50K'}), frozenset({'<=50K', 'White'}), 0.8373381877022654],
 [frozenset({'<=50K'}),
  frozenset({'<=50K', 'United-States'}),
  0.8899271844660194],
 [frozenset({'0'}), frozenset({'0', 'Private'}), 0.6970301894904949],
 [frozenset({'0'}), frozenset({'0', 'White'}), 0.8542735173981143],
 [frozenset({'0'}), frozenset({'0', 'Male'}), 0.6692054912318418],
 [frozenset({'0'}), frozenset({'0', 'United-States'}), 0.8958570068486841],
 [frozenset({'0'}), frozenset({'0', '<=50K'}), 0.7591904425539756],
 [frozenset({'United-States'}),
  frozenset({'United-States', 'White'}),
  0.8783339046966061],
 [frozenset({'Male'}), frozenset({'Male', 'White'}), 0.8799449288664526],
 [frozenset({'Male'}),
  frozenset({'Male', 'United-States'}),
  0.

In [9]:
time1 = time.perf_counter()
L, C, rule = apriori(data,0.3,0.6)
time2 = time.perf_counter()
print("Running time when minSup is 0.3: ", time2 - time1)

Running time when minSup is 0.3:  214.98105309999997


In [10]:
time1 = time.perf_counter()
L, C, rule = apriori(data,0.8,0.6)
time2 = time.perf_counter()
print("Running time when minSup is 0.8: ", time2 - time1)

Running time when minSup is 0.8:  193.30807030000005


FP-Growth

https://www.cnblogs.com/ybjourney/p/4851540.html gives the idea.

In [15]:
class Node:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue 
        self.count = numOccur
        self.nodeLink = None # To connect node with same name in FPtree
        self.parent = parentNode
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

In [16]:
def update_nodelink(nodeWillAdd, nodeToAdd):
    """
    Goal: Update nodeLink in FPtree
    ------
    Parameter: 
        nodeWillAdd: the first node with the same name as nodeToAdd
        nodeToAdd: node shoule be linked with other nodes with the same node name
    """
    while nodeWillAdd.nodeLink != None:
        nodeWillAdd = nodeWillAdd.nodeLink
    nodeWillAdd.nodeLink = nodeToAdd

In [17]:
def update_fptree(items, node, headerTable):
    """
    Goal: Update FPtree
    ------
    Parameter: 
        items: the list of items needed to be updated to the FP tree
        node: the current node needed to check with items
        headerTable: the table with item and their support, all supports of items >= minSup
    """
    if items[0] in node.children:# Just update count
            node.children[items[0]].count+=1
    else:
        # Create a new node
        node.children[items[0]] = Node(items[0],1,node)
        # Right now headertable of this item does not link to any node in the fp tree, update it
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = node.children[items[0]]
        else:
            # HeaderTable already have linked with one node in the fp tree, update the nodelink of this linked node
            update_nodelink(headerTable[items[0]][1], node.children[items[0]])
    if len(items) > 1:
        update_fptree(items[1:], node.children[items[0]], headerTable)

In [18]:
def create_fptree(dataset,minSup):
    """
    Goal: create FP Tree
    ------
    Parameter: 
        dataset: input data
        minSup: minimum support that item needs to satisfy
    ------
    Return: 
        tree_header: The root node of FP Tree
        headerTable: The item in the dataset and their support, where support of items all >= minSup
    """
    item_count = {}
    # First time iteration, construct item with support >= minSup
    for trans in dataset:
        for item in trans:
            if item not in item_count:
                item_count[item]=1
            else:
                item_count[item]+=1
    
    headerTable = {}
    freqItem = set()
    for k in item_count:
        if item_count[k] >= minSup:
            headerTable[k] = item_count[k]
            freqItem.add(k)      
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    tree_header = Node('null',1,None)
    # Second time iteration
    for t in dataset:
        tmp = {}
        for item in t:
            if item in freqItem:
                tmp[item] = headerTable[item][0] # retrieve the count of the item
        if len(tmp) > 0:
            ordered = [v[0] for v in sorted(tmp.items(), key=lambda x:x[1], reverse=True)]
            update_fptree(ordered,tree_header,headerTable)
    return tree_header, headerTable

In [19]:
def generate_rule(HeaderTab, minConf = 0.6):
    """
    Goal: generate rules with confidence >= minConf
    ------
    Parameter: 
       strong_rule: rules with confidence >= minConf
    ------
    Return: 
        tree_header: The root node of FP Tree
        headerTable: The item in the dataset and their support, where support of items all >= minSup
    """
    HT = sorted(HeaderTab.items(), key=lambda x:x[1][0])
    rule = {}
    for i in range(len(HT)):
        r = HT[i][1][1]
        while r:
            tmp = r.name
            times = r.count
            parent = r.parent
            while parent.name != 'null':
                tmp = parent.name +','+ tmp
                if tmp in rule:
                    rule[tmp] += times
                else:
                    rule[tmp] = times
                parent = parent.parent
            r = r.nodeLink
    strong_rule = []
    for i in rule:
        if len(i.split(',')) == 2:
            if rule[i] / HeaderTab[i.split(',')[0]][0] >= minConf:
                strong_rule.append([i.split(',')[0],i,rule[i] / HeaderTab[i.split(',')[0]][0]])
        else:
            tmp = i.split(',')[:-1]
            prev = tmp[0]
            for j in tmp[1:]:
                prev = prev +','+ j
            if prev in rule:
                if rule[i] / rule[prev] >= minConf:
                    strong_rule.append([prev,i,rule[i] / rule[prev]])
    return strong_rule

In [26]:
time1 = time.perf_counter()
FPtree, HeaderTab = create_fptree(data, len(data)//2)
rule = generate_rule(HeaderTab,0.6)
time2 = time.perf_counter()
print("Running time is: ", time2 - time1)

Running time is:  0.18037259999999833


In [27]:
rule

[['Private', 'Private,Male', 0.6584420162143109],
 ['<=50K,Private', '<=50K,Private,Male', 0.6037895449162578],
 ['White,<=50K,Private', 'White,<=50K,Private,Male', 0.6206293706293706],
 ['United-States,White,<=50K,Private',
  'United-States,White,<=50K,Private,Male',
  0.6155961938745168],
 ['0,United-States,White,<=50K,Private',
  '0,United-States,White,<=50K,Private,Male',
  0.6155961938745168],
 ['White,Private', 'White,Private,Male', 0.8590026478375993],
 ['United-States,White,Private',
  'United-States,White,Private,Male',
  0.8594480823199252],
 ['0,United-States,White,Private',
  '0,United-States,White,Private,Male',
  0.8594480823199252],
 ['United-States,Private', 'United-States,Private,Male', 0.769811320754717],
 ['0,United-States,Private',
  '0,United-States,Private,Male',
  0.769811320754717],
 ['0,Private', '0,Private,Male', 0.8433734939759037],
 ['0,White,<=50K,Private', '0,White,<=50K,Private,Male', 0.6683098591549296],
 ['0,White,Private', '0,White,Private,Male', 0.851

In [18]:
time1 = time.perf_counter()
FPtree, HeaderTab = create_fptree(data, 3*len(data)//10)
rule = generate_rule(HeaderTab,0.6)
time2 = time.perf_counter()
print("Running time of minSup 0.3: ", time2 - time1)

Running time of minSup 0.3:  0.23030479999999898


In [36]:
time1 = time.perf_counter()
FPtree, HeaderTab = create_fptree(data, 8*len(data)//10)
rule = generate_rule(HeaderTab,0.6)
time2 = time.perf_counter()
print("Running time of minSup 0.8: ", time2 - time1)

Running time of minSup 0.8:  0.12676319999991392


Ecalt

https://blog.csdn.net/sinat_35821976/article/details/90744559 gives the idea.

In [20]:
def firstScan(data, minSup=3):
    item_row = {}
    for i in range(len(data)):
        for item in data[i]:
            if item in item_row:
                item_row[item].append(i)
            else:
                item_row[item] = [i]
    
    freqItem = []
    support = []
    for item in item_row.keys():
        if len(item_row[item]) >= minSup:
            freqItem.append([item])
            support.append(item_row[item])
    return freqItem, support

In [21]:
def generate_k(freqItem, support, minSup=3):
    interFreq = []
    interSup = []
    k = len(freqItem[0])
    for i in range(len(freqItem)):
        for j in range(i+1,len(freqItem)):
            L1 = list(freqItem[i])[:k-1]
            L2 = list(freqItem[j])[:k-1]
            if L1 == L2:
                sup = len(list(set(support[i]).intersection(set(support[j]))))
                if sup >= minSup:
                    interFreq.append(list(set(freqItem[i])|set(freqItem[j])))
                    interSup.append(list(set(support[i]).intersection(set(support[j]))))
    return interFreq, interSup

In [22]:
def KfreqItem(freqItem, support, freqSet, supportSet,minSup=3):
    freqSet.append(freqItem)
    supportSet.append(support)
    while len(freqItem) >= 2:
        kFreq, kSup = generate_k(freqItem,support,minSup)
        freqSet.append(kFreq)
        supportSet.append(kSup)
        freqItem = kFreq.copy()
        support = kSup.copy()
    return freqSet,supportSet

In [23]:
def generate_rule(freqItem, support,minConf=0.6):
    rule = []
    for i in range(len(freqItem)-1,0,-1):
        for j in range(len(freqItem[i])):
            for z in range(len(freqItem[i-1])):
                if freqItem[i][j][:-1] == freqItem[i-1][z]:
                    if (len(support[i][j]) / len(support[i-1][z])) >= minConf:
                        rule.append([freqItem[i-1][z],freqItem[i][j],len(support[i][j]) / len(support[i-1][z])])
    return rule

In [24]:
time1 = time.perf_counter()
a,b = firstScan(data)
c,d = KfreqItem(a,b,[],[],len(data)//2) 
rule = generate_rule(c, d,0.6)
time2 = time.perf_counter()
print("Running time is: ", time2 - time1)

Running time is:  35.27344190000008


In [25]:
rule

[[['Male', 'White'], ['Male', 'White', '0'], 1.0],
 [['Male', 'White'], ['Male', 'White', 'United-States'], 0.920673829143632],
 [['0', 'White'], ['0', 'White', 'United-States'], 0.9210885821110153],
 [['0', '<=50K'], ['0', '<=50K', 'White'], 0.8373381877022654],
 [['0', 'Private'], ['0', 'Private', 'White'], 0.8549524145223828],
 [['<=50K', 'White'], ['<=50K', 'White', 'United-States'], 0.9139088844871733],
 [['Private', 'White'],
  ['Private', 'White', 'United-States'],
  0.9136260564831994],
 [['Male', 'United-States'], ['Male', 'United-States', '0'], 1.0],
 [['0', '<=50K'], ['0', '<=50K', 'United-States'], 0.8899271844660194],
 [['0', 'Private'], ['0', 'Private', 'United-States'], 0.8871607331688404],
 [['0', '<=50K'], ['0', '<=50K', 'Private'], 0.7173543689320389],
 [['Male'], ['Male', 'White'], 0.8799449288664525],
 [['White'], ['White', 'United-States'], 0.9210885821110153],
 [['<=50K'], ['<=50K', 'White'], 0.8373381877022654],
 [['Private'], ['Private', 'White'], 0.854952414522

In [26]:
time1 = time.perf_counter()
a,b = firstScan(data)
c,d = KfreqItem(a,b,[],[],3*len(data)//10) 
rule = generate_rule(c, d,0.6)
time2 = time.perf_counter()
print("Running time with minSup 0.3: ", time2 - time1)

Running time with minSup 0.3:  35.446755399999915


In [27]:
time1 = time.perf_counter()
a,b = firstScan(data)
c,d = KfreqItem(a,b,[],[],8*len(data)//10) 
rule = generate_rule(c, d,0.6)
time2 = time.perf_counter()
print("Running time with minSup 0.8: ", time2 - time1)

Running time with minSup 0.8:  36.21804840000004


Conclusion:
    FP Growth uses less time than Ecalt and Apriori, and Apriori uses the most time among these three.
    Because FP Growth only needs to scan the dataset twice.
    If we set minSup to a small value, Ecalt will take super long time to run, but it still has a better performance than Apriori.
    Whatever min support we set, FP Growth has a similiar fast speed.

Improvement: 
    Apriori cannot deal with multiple occureance in each transaction.
    For example, in the hw, 'COOKIE', apriori will only count 'O' once, and not consider 'OO'.
    The improvement is to modify it to consider multiple occurance as an item.
    I do not think Adult dataset has such a thing, so I modify the one on the textbook.

In [29]:
data = load_data_set()

In [30]:
def load_data_set():
    """
    Load a sample data set (From Data Mining: Concepts and Techniques, 3th Edition)
    Returns: 
        A data set: A list of transactions. Each transaction contains several items.
    """
    data_set = [['l1', 'l2','l2', 'l5'], ['l2','l2', 'l4'], ['l2','l2', 'l3'],
            ['l1', 'l2', 'l4','l2'], ['l1', 'l3','l2'], ['l2', 'l3','l2'],
            ['l1', 'l3'], ['l1', 'l2', 'l3', 'l5'], ['l1', 'l2','l2', 'l3']]
    return data_set

In [31]:
def create_C1(dataset):
    """
    Goal: generate C1
    ------
    Parameter: 
        dataset: input dataset
    ------
    Return:
        C1: set with all distinct elements
    """

    C1 = set()
    for t in dataset:
        for item in t:
            item_set = frozenset([item])
            if item_set in C1:
                item_set = frozenset([item+','+item])
            C1.add(item_set)
    return C1

In [92]:
support_count = {}
for itemset in data:
    for candidate in c1:
        belong = True
        tmp = list(candidate)[0].split(',')
        for i in tmp:
            if not i in itemset:
                belong = False
        if belong:
            if candidate in support_count:
                support_count[candidate] += 1
            else:
                support_count[candidate] = 1

In [36]:
def create_L(D,Ck,minSupport):
    """
    Goal: count support of item in Ck, and filter items
    ------
    Parameter: 
        D: input dataset
        Ck: C generated by either create_C1 or apriori_Gen, k indicates the length of item
        minSupport: threshold of support
    ------
    Return:
        Lk: the filtered candidate with length k, which support >= min support
        supportData: the candidate with support
    """
    support_count = {}
    for itemset in D:
        for candidate in Ck:
            belong = True
            tmp = list(candidate)[0].split(',')
            for i in tmp:
                if not i in itemset:
                    belong = False
            if belong:
                if candidate in support_count:
                    support_count[candidate] += 1
                else:
                    support_count[candidate] = 1
    Lk = []
    supportData = {}
    for key in support_count:
        support = support_count[key] / float(len(D))
        if support >= minSupport:
            Lk.insert(0, key)
        supportData[key] = support
    return Lk, supportData

In [38]:
def create_Ck(Lk, k):
    """
    Goal: create C set with length k
    ------
    Parameter: 
        Lk: valid items with support
        k: length of item in output Ck
    ------
    Return:
        Ck: candidate itemset, for item in Ck, length(item) = k
    """
    Ck = []
    for i in range(len(Lk)):
        for j in range(i+1,len(Lk)):
            # Check first K-2 same or not
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2: # If same, then combine these 2 to generate new one
                Ck.append(Lk[i] | Lk[j])
    return Ck

In [39]:
def apriori(dataset, minSupport = 0.5, minConf = 0.6):
    """
    Goal: Find strong rule in dataset
    ------
    Parameter: 
        dataset: input dataset
        minSupport: min support that a item needs to satisfy
        minConf: min confidence that a rule needs to satisfy
    ------
    Return:
        L: itemset with support >= minSupport
        B: itemset with support
        rule: rule with confidence >= minConf
    """
    C1 = create_C1(dataset)
    L1, support = create_L(dataset, C1, minSupport)
    L = [L1]
    k = 2 # We already have L1, start from 2
    rule = []
    while len(L[k-2])>0:
        Ck = create_Ck(L[k-2], k)
        Lk, supportK = create_L(dataset, Ck, minSupport)
        # Each time we have a new support for length k, we will try to find the strong rule
        for i in L[len(L)-1]:
            for j in Lk:
                Anced = True # Check whether this rule is the parent of another
                for z in zip(i,j):
                    if z[0] != z[1]:
                        Anced = False
                if Anced:
                    if (supportK[j]/support[i]) >= minConf:
                        rule.append([i,j,supportK[j]/support[i]]) # Parent, Child, Conf    
        support.update(supportK)
        L.append(Lk)
        k += 1
    return L, support, rule

In [44]:
L, C, rule = apriori(data,0.5,0.6)

In [41]:
rule

[[frozenset({'l3,l3'}), frozenset({'l2,l2', 'l3,l3'}), 1.0],
 [frozenset({'l3,l3'}), frozenset({'l1', 'l3,l3'}), 1.0],
 [frozenset({'l3,l3'}), frozenset({'l1,l1', 'l3,l3'}), 1.0],
 [frozenset({'l3,l3'}), frozenset({'l2', 'l3,l3'}), 1.0],
 [frozenset({'l3,l3'}), frozenset({'l3', 'l3,l3'}), 1.0],
 [frozenset({'l3'}), frozenset({'l2,l2', 'l3'}), 1.0],
 [frozenset({'l3'}), frozenset({'l1', 'l3'}), 1.0],
 [frozenset({'l3'}), frozenset({'l1,l1', 'l3'}), 1.0],
 [frozenset({'l3'}), frozenset({'l2', 'l3'}), 1.0],
 [frozenset({'l2'}), frozenset({'l1,l1', 'l2'}), 1.0],
 [frozenset({'l1'}), frozenset({'l1', 'l1,l1'}), 1.0],
 [frozenset({'l1'}), frozenset({'l1', 'l2'}), 1.0],
 [frozenset({'l2,l2'}), frozenset({'l1', 'l2,l2'}), 1.0],
 [frozenset({'l2,l2'}), frozenset({'l1,l1', 'l2,l2'}), 1.0],
 [frozenset({'l2,l2'}), frozenset({'l2', 'l2,l2'}), 1.0],
 [frozenset({'l2,l2', 'l3'}), frozenset({'l2', 'l2,l2', 'l3'}), 1.0],
 [frozenset({'l2,l2', 'l3'}), frozenset({'l1,l1', 'l2,l2', 'l3'}), 1.0],
 [frozen